In [29]:
import pandas as pd
import numpy as np
import scipy

from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures

import statsmodels.api as sm

import matplotlib.pyplot as plt

import os
import datetime

### Read Data

In [30]:
# Read in the bid-ask data from WRDS
bid_ask_data = pd.read_sas('../Data/dataset.sas7bdat')
bid_ask_data.head()

,PERMNO,DATE,BID,ASK,BIDLO,ASKHI,VOL,PRC,decile_category,stock_num
0,10514.00,2000-01-03,10.44,10.56,10.12,10.50,368119.00,10.50,1.00,1.00
1,10661.00,2000-01-03,1.62,1.66,1.38,1.81,52683.00,1.66,1.00,2.00
2,10886.00,2000-01-03,7.12,7.44,7.12,7.75,26362.00,7.25,1.00,3.00
3,11394.00,2000-01-03,2.38,2.44,2.28,2.38,30025.00,2.38,1.00,4.00
4,15203.00,2000-01-03,3.62,3.75,3.62,3.75,0.00,-3.69,1.00,5.00


In [31]:
bid_ask_data.describe()

,PERMNO,BID,ASK,BIDLO,ASKHI,VOL,PRC,decile_category,stock_num
count,996827.00,996827.00,996827.00,996827.00,996827.00,996827.00,996827.00,996827.00,996827.00
mean,32848.49,27.98,28.15,27.65,28.46,1064732.12,27.18,5.47,10.50
std,24056.47,41.60,41.83,41.26,42.15,5688094.30,42.30,2.86,5.77
min,10026.00,0.01,0.03,0.01,0.03,0.00,-362.77,1.00,1.00
25%,11481.00,7.38,7.49,7.25,7.60,9962.50,6.82,3.00,5.00
50%,21259.00,17.65,17.82,17.39,18.00,57508.00,17.30,5.00,11.00
75%,51530.00,33.56,33.71,33.11,34.10,285085.50,33.44,8.00,16.00
max,78213.00,919.58,933.70,920.05,948.03,592924962.00,927.00,10.00,20.00


In [32]:
# Read in the CCI data
CCI_data = pd.read_csv('../Data/CCI.csv')
CCI_data.head()

,LOCATION,INDICATOR,SUBJECT,MEASURE,FREQUENCY,TIME,Value,Flag Codes
0,NLD,CCI,AMPLITUD,LTRENDIDX,M,1991-01,99.76,nan
1,NLD,CCI,AMPLITUD,LTRENDIDX,M,1991-02,99.75,nan
2,NLD,CCI,AMPLITUD,LTRENDIDX,M,1991-03,99.89,nan
3,NLD,CCI,AMPLITUD,LTRENDIDX,M,1991-04,99.96,nan
4,NLD,CCI,AMPLITUD,LTRENDIDX,M,1991-05,100.01,nan


In [33]:
# Read in the ICS data
ICS_data = pd.read_csv('../Data/ICS.csv')
ICS_data.head()

,Month,YYYY,ICS_ALL
0,November,1952,86.20
1,February,1953,90.70
2,August,1953,80.80
3,November,1953,80.70
4,February,1954,82.00


### Helper Functions

In [34]:
def bid_ask_estimate(df, id_col='PERMNO', date_col='DATE', high_col='ASKHI', low_col='BIDLO'):
    """
    Inputs:
     - df [DataFrame]: contains all data
     - id_col [str]: name of stock identifying column
     - date_col [str]: name of date column
     - high_col [str]: name of column for high price
     - low_col [str]: name of column for low price
    
    Return: modifies df input to contain the Corwin-Schultz estimate of the bid-ask spread
    """
    
    temp = bid_ask_df.copy()
    temp.sort_values(['PERMNO', 'DATE'])
    temp["next_H"] = temp.groupby(['PERMNO'])['ASKHI'].shift(-1) # Get high for next day
    temp["next_L"] = temp.groupby(['PERMNO'])['BIDLO'].shift(-1) # get low for next day
    
    # beta = E{sum(j=0,1)[ln(H_(t+j)/L_(t+j))]^2} where H_t is the observed high price on day t (similarly for L)
    beta = (np.log(temp[high_col]/temp[low_col])**2 + np.log(temp.next_H/temp.next_L)**2)
    
    # gamma = [ln(H_(t, t+1)/L_(t, t+1))]^2 where H_(t, t+1) is the high price over days t and t+1 (similarly for L)
    gamma = (np.log(np.maximum(temp[high_col], temp.next_H)/np.minimum(temp[low_col], temp.next_L)))**2
    
    # alpha = ((2beta)^0.5-beta^0.5)/(3-2*2^0.5) - (gamma/(3-2*2^0.5))^0.5
    alpha = (np.sqrt(2*beta)-np.sqrt(beta))/(3-2*np.sqrt(2))-np.sqrt(gamma/(3-2*np.sqrt(2)))
    
    # high-low spread estimate S = 2(e^alpha - 1)/(1+e^alpha)
    S = 2*(np.exp(alpha)-1)/(1+np.exp(alpha))
    
    temp['CW_spread_estimate'] = S
    
    return temp

In [35]:
def split_date_field(df, date_field):
    '''
    Extracts year and month columns from date column
    
    Return: df with month and year columns
    '''
    new_df = df.copy()
    
    split_date = new_df[date_field].str.split('-', n=1, expand=True)
    
    new_df['Year'] = split_date[0].astype(int)
    new_df['Month'] = split_date[1].astype(int)
    
    return new_df
    
def get_integer_month(df, month_field, short_names=False):
    '''
    Get months as integers from string month field
    
    Return: df with updated month field to have integer values
    '''
    
    new_df = df.copy()
    
    # Check for integer months
    if pd.api.types.is_integer_dtype(df[month_field]):
        print("Month field already in integer form.")
        return df
    
    # If not integer months, convert months to integers
    if short_names:
        new_df[month_field] = new_df[month_field].apply(lambda x: datetime.datetime.strptime(x, "%b").month)
    
    else:
        new_df[month_field] = new_df[month_field].apply(lambda x: datetime.datetime.strptime(x, "%B").month)

    return new_df

In [36]:
def get_daily_data(df, trading_days, year_field, month_field, cols_to_keep=None):
    '''
    Interpolates the data on a daily basis from a monthly basis (same value between dates)
    
    Return: df with daily data
    '''
    
    trading_months = trading_days.month
    trading_years = trading_days.year
    date_df = pd.DataFrame({'Date': trading_days, 'Year': trading_years, 'Month': trading_months})
    
    if cols_to_keep:
        merged_df = date_df.merge(df[cols_to_keep+[year_field]+[month_field]], left_on=['Year', 'Month'], 
                                  right_on=[year_field, month_field], how='left')
    
    else:
        merged_df = date_df.merge(df, left_on=['Year', 'Month'], 
                                  right_on=[year_field, month_field], how='left')        
    
    return merged_df

## Pre-process the Data

### (0) Australian Data

In [37]:
# Australian confidence index
CCI_data_AUS = CCI_data[CCI_data.LOCATION == 'AUS']
CCI_data_AUS = split_date_field(CCI_data_AUS, 'TIME')
CCI_data_AUS = CCI_data_AUS.groupby(['TIME']).mean()[['Value', 'Year', 'Month']].reset_index()

### (i) CCI & ICS data

In [38]:
# Extract month and year, then average the CCI data across each state
CCI_data_cleaned = CCI_data[CCI_data.LOCATION == 'USA']
CCI_data_cleaned = split_date_field(CCI_data_cleaned, 'TIME')
CCI_data_cleaned = CCI_data_cleaned.groupby(['TIME']).mean()[['Value', 'Year', 'Month']].reset_index()

# Get the integer monthly values for ICS data
ICS_data_cleaned = get_integer_month(ICS_data, 'Month')

# Get trading days
trading_days = pd.to_datetime(bid_ask_data.DATE.unique().astype(str))

# Get daily data for CCI and ICS
CCI_daily = get_daily_data(CCI_data_cleaned, trading_days, 'Year', 'Month')
ICS_daily = get_daily_data(ICS_data_cleaned, trading_days, 'YYYY', 'Month')

# Get rid of unnecessary columns, rename columns
CCI_daily = CCI_daily.drop(['Year', 'TIME', 'Month'], axis=1)
CCI_daily = CCI_daily.rename({'Value': 'CCI'}, axis='columns')

ICS_daily = ICS_daily.drop(['Year', 'YYYY', 'Month'], axis=1)
ICS_daily = ICS_daily.rename({'ICS_ALL': 'ICS'}, axis='columns')

### (ii) bid-ask data

bid_ask_df: holds stock-level information

bid_ask_deciles: holds decile-level information

In [39]:
bid_ask_df = bid_ask_data.copy()

#### Spread Calculation

Calculated on stock level.
Result (ABS_SPREAD, REL_SPREAD) added to bid_ask_df table.

In [40]:
# Convert date field to datetime
bid_ask_df['DATE'] = pd.to_datetime(bid_ask_df.DATE.astype(str))

# Get the absolute and relative bid-ask spread data
bid_ask_df['ABS_SPREAD'] = bid_ask_df.ASK - bid_ask_df.BID
bid_ask_df['REL_SPREAD'] = (bid_ask_df.ASK - bid_ask_df.BID)*2/(bid_ask_df.ASK + bid_ask_df.BID)

In [41]:
bid_ask_df[bid_ask_df.REL_SPREAD < 0]

,PERMNO,DATE,BID,ASK,BIDLO,ASKHI,VOL,PRC,decile_category,stock_num,ABS_SPREAD,REL_SPREAD


#### Volume Calculation

Calculated on stock level.
Result (DLR_VOL) added to bid_ask_df table.

In [42]:
import warnings
warnings.filterwarnings("ignore")

# Get relevant columns
bid_ask_subset = bid_ask_df[['PERMNO', 'DATE', 'decile_category', 'VOL', 'PRC']].copy()

# Get the dollar volume for the previous day for each stock
i = 1
for permno, group in bid_ask_subset.groupby('PERMNO'):
    group.sort_values('DATE')
    group['PRC_shifted'] = group['PRC'].shift(1) # Shifted price
    group['DLR_shifted'] = group['VOL'].shift(1) # Shifted volume
    group['DLR_VOL'] = group.PRC_shifted*group.DLR_shifted # Previous day's dollar volume

    if i == 1:
        bid_ask_vol = group
    else:
        bid_ask_vol = pd.concat([bid_ask_vol, group])
        
    i += 1
    
# Save results to bid_ask_df table
bid_ask_df['DLR_VOL'] = bid_ask_vol['DLR_VOL'] 

#### Return and Variance Calculation

Calculated on stock level.
Result (DLY_RETURN) added to bid_ask_df table.

In [43]:
import warnings
warnings.filterwarnings("ignore")

bid_ask_subset = bid_ask_df[['DATE', 'PERMNO', 'decile_category', 'PRC']].copy()

# Get the individual stock returns for each stock
i = 1
for permno, group in bid_ask_subset.groupby('PERMNO'):
    group.sort_values('DATE')
    group['PRC_shifted'] = group['PRC'].shift(1) # Shifted price
    group['daily_return'] = (abs(group.PRC)-abs(group.PRC_shifted))/abs(group.PRC_shifted) # Return
    
    if i == 1:
        bid_ask_return = group
    else:
        bid_ask_return = pd.concat([bid_ask_return, group])
    i += 1

bid_ask_df['DLY_RETURN'] = bid_ask_return['daily_return'] # Save to bid_ask_df table

#### Variance Calculation

Calculated on stock level.
Result (DLY_VARIANCE) added to bid_ask_df table.

In [44]:
# Remove NaN rows
bid_ask_return = bid_ask_return.dropna()

# Get the variances based on 21-day periods for each stock
period_length = 21
i = 1
for permno, group in bid_ask_return.groupby('PERMNO'):
    group.sort_values('DATE')
    group['variance'] = group.daily_return.rolling(period_length).var()
    
    if i == 1:
        bid_ask_var = group
    else:
        bid_ask_var = pd.concat([bid_ask_var, group])
    i += 1

bid_ask_df['DLY_VARIANCE'] = bid_ask_var['variance'] # Save to bid_ask_df table

In [45]:
bid_ask_df = bid_ask_df.dropna() # Remove NaN values

#### Detrend Calculations on Decile Level

##### Aggregate Spread Data

Calculated on decile level. Result (ABS_SPREAD, REL_SPREAD) saved to bid_ask_deciles.

In [46]:
# Get the equal-weighted averages of firm-specific daily closing quoted spreads
bid_ask_deciles = bid_ask_df[['DATE', 'decile_category', 'ABS_SPREAD', 'REL_SPREAD']].groupby(['DATE', 'decile_category']).mean().reset_index()

##### Detrend Volume and Variance

Calculated on decile level.
Result (DLR_VOL_detrended, VAR_detrended) added to bid_ask_deciles table.

In [47]:
# Get average dollar volume and variance across each decile
bid_ask_grouped = bid_ask_df[['DATE', 'decile_category', 'DLR_VOL', 'DLY_VARIANCE']].groupby(['DATE', 'decile_category']).mean().reset_index()

# Calculate average (volume, variance) over last 252 days for each decile
for i, group in bid_ask_grouped.groupby('decile_category'):
    group.sort_values('DATE')
    group['AVG_DLR_VOL'] = group.DLR_VOL.rolling(252).mean()
    group['AVG_VAR'] = group.DLY_VARIANCE.rolling(252).mean()
    
    if i == 1:
        averaged_data = group
    else:
        averaged_data = pd.concat([averaged_data, group])
        
# Calculate the detrended volume and variance
averaged_data['DLR_VOL_detrended'] = averaged_data.DLR_VOL/averaged_data.AVG_DLR_VOL
averaged_data['VAR_detrended'] = averaged_data.DLY_VARIANCE/averaged_data.AVG_VAR

# Save to bid_ask_deciles table
bid_ask_deciles['DLR_VOL_detrended'] = averaged_data['DLR_VOL_detrended'] 
bid_ask_deciles['VAR_detrended'] = averaged_data['VAR_detrended'] 

In [48]:
# Remove NaN rows
bid_ask_deciles = bid_ask_deciles.dropna().rename(columns={'DATE':'Date'})
bid_ask_deciles

,Date,decile_category,ABS_SPREAD,REL_SPREAD,DLR_VOL_detrended,VAR_detrended
2510,2001-01-31,1.00,0.16,0.03,0.37,0.79
2511,2001-01-31,2.00,0.23,0.04,0.64,0.83
2512,2001-01-31,3.00,0.23,0.03,0.58,1.03
2513,2001-01-31,4.00,0.30,0.03,0.29,1.54
2514,2001-01-31,5.00,0.90,0.02,0.54,0.92
...,...,...,...,...,...,...
50095,2019-12-31,6.00,0.13,0.00,0.74,30.42
50096,2019-12-31,7.00,0.84,0.00,0.72,0.49
50097,2019-12-31,8.00,0.04,0.00,0.82,0.31
50098,2019-12-31,9.00,0.15,0.00,0.70,0.59


### (iii) combine data

In [49]:
# Combine datasets into two dataframes: bid_ask_deciles and CCI_daily, ICS_daily
all_data = bid_ask_deciles.merge(CCI_daily, on=['Date'], how='left')
all_data = all_data.merge(ICS_daily, on=['Date'], how='left')
all_data = all_data.set_index('Date')

## Get Summary Statistics

In [50]:
pd.set_option('float_format', '{:.2f}'.format)

# Summary over all US data
summary_cols = ['Quote Spread (%)', 'Bid ($)', 'Ask ($)', 'Volume ($million)', 'Daily Return (%)', 'Return Variance (%)']

bid_ask_df['Quote Spread (%)'] = bid_ask_df.REL_SPREAD*100
bid_ask_df['Bid ($)'] = bid_ask_df.BID
bid_ask_df['Ask ($)'] = bid_ask_df.ASK
bid_ask_df['Volume ($million)'] = bid_ask_df.DLR_VOL/1000000
bid_ask_df['Daily Return (%)'] = bid_ask_df.DLY_RETURN*100
bid_ask_df['Return Variance (%)'] = bid_ask_df.DLY_VARIANCE*100

bid_ask_df[summary_cols].describe().transpose()[['mean', 'std', 'min', 'max']]

,mean,std,min,max
Quote Spread (%),1.20,2.51,0.00,198.37
Bid ($),28.01,41.66,0.01,919.58
Ask ($),28.18,41.89,0.03,933.70
Volume ($million),48.69,419.40,-1.40,30741.17
Daily Return (%),0.14,40.45,-89.52,39344.44
Return Variance (%),16.34,3391.15,0.00,737246.06


In [23]:
pd.set_option('float_format', '{:.2f}'.format)

# Summary over US decile data
summary_cols = ['Quote Spread (%)', 'Detrended Volume (%)', 'Detrended Variance (%)']

bid_ask_deciles['Quote Spread (%)'] = bid_ask_deciles.REL_SPREAD*100
bid_ask_deciles['Detrended Volume (%)'] = bid_ask_deciles.DLR_VOL_detrended*100
bid_ask_deciles['Detrended Variance (%)'] = bid_ask_deciles.VAR_detrended*100

for i, group in bid_ask_deciles.groupby('decile_category'):
    temp = group[summary_cols].describe().transpose()[['mean', 'std', 'min', 'max']]
    temp['Decile'] = i
    
    if i == 1:
        summary_df = temp
    else:
        summary_df = pd.concat([summary_df, temp])

summary_df.index.name = 'Variable'
summary_df = summary_df.reset_index().set_index(['Decile','Variable'])

In [24]:
summary_df

mean    std   min      max
Decile Variable                                           
1.00   Quote Spread (%)         2.52   1.32  0.59    22.98
       Detrended Volume (%)    99.98 148.93  7.76  5418.16
       Detrended Variance (%) 144.73 549.25  0.65 18014.77
2.00   Quote Spread (%)         3.03   1.51  0.89    29.67
       Detrended Volume (%)   105.20 117.95 11.00  2923.75
       Detrended Variance (%) 103.03 121.56 14.32  2918.82
3.00   Quote Spread (%)         1.49   0.95  0.33    21.76
       Detrended Volume (%)   101.33  84.48  7.39  1934.28
       Detrended Variance (%) 149.80 753.72  0.00 25169.74
4.00   Quote Spread (%)         1.38   0.75  0.28    12.50
       Detrended Volume (%)   102.33  74.90 12.85  1423.93
       Detrended Variance (%) 123.75 454.91  0.14 13419.03
5.00   Quote Spread (%)         1.24   0.82  0.04    23.18
       Detrended Volume (%)   103.64  52.57 18.60   815.31
       Detrended Variance (%)  99.35  68.52 24.33   875.22
6.00   Quote Spread (%)         0.52   0.47  0.08     5.91
       Detrended Volume (%)   104.36  53.40 16.13   975.95
       Detrended Variance (%) 113.03 368.67 21.28 19322.19
7.00   Quote Spread (%)         0.51   0.58  0.04    12.27
       Detrended Volume (%)   103.74  42.53 21.32   570.72
       Detrended Variance (%) 110.11 223.00  4.72  5643.47
8.00   Quote Spread (%)         0.24   0.34 -0.27     9.73
       Detrended Volume (%)   103.59  34.59 19.09   376.29
       Detrended Variance (%)  98.55  53.57 16.83   430.80
9.00   Quote Spread (%)         0.18   0.19 -0.19     6.49
       Detrended Volume (%)   105.45  32.51 22.53   434.63
       Detrended Variance (%) 110.41 304.01  0.61 13794.45
10.00  Quote Spread (%)         0.06   0.08 -0.35     2.43
       Detrended Volume (%)   102.29  32.75 15.10   328.43
       Detrended Variance (%) 100.82  77.34 13.06   664.04

#### Investigating High Variance

In [28]:
# bid_ask_df[bid_ask_df.DLY_VARIANCE > 30]
# test = bid_ask_df[bid_ask_df.PERMNO == 59483]
# test[test.DATE <= '2011-12-07']


test = bid_ask_df[bid_ask_df.PERMNO == 76858]
test[test.DATE <= '2021-02-02']


,PERMNO,DATE,BID,ASK,BIDLO,ASKHI,VOL,PRC,decile_category,stock_num,...,REL_SPREAD,DLR_VOL,DLY_RETURN,DLY_VARIANCE,Quote Spread (%),Bid ($),Ask ($),Volume ($million),Daily Return (%),Return Variance (%)
302296,76858.00,2000-02-02,1.28,1.41,1.22,1.41,37830.00,1.31,4.00,17.00,...,0.09,2625.00,0.00,0.01,9.30,1.28,1.41,0.00,0.00,0.62
302316,76858.00,2000-02-03,1.28,1.31,1.28,1.38,8400.00,1.28,4.00,17.00,...,0.02,49651.88,-0.02,0.01,2.41,1.28,1.31,0.05,-2.38,0.51
302336,76858.00,2000-02-04,1.28,1.31,1.28,1.38,7549.00,1.31,4.00,17.00,...,0.02,10762.50,0.02,0.00,2.41,1.28,1.31,0.01,2.44,0.45
302356,76858.00,2000-02-07,1.28,1.31,1.25,1.41,54240.00,1.31,4.00,17.00,...,0.02,9908.06,0.00,0.00,2.41,1.28,1.31,0.01,0.00,0.40
302376,76858.00,2000-02-08,1.28,1.38,1.28,1.31,57750.00,1.28,4.00,17.00,...,0.07,71190.00,-0.02,0.00,7.06,1.28,1.38,0.07,-2.38,0.39
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
402396,76858.00,2019-12-24,2.15,2.21,2.10,2.58,54799.00,2.20,4.00,17.00,...,0.03,127919.18,-0.14,0.00,2.75,2.15,2.21,0.13,-14.40,0.41
402416,76858.00,2019-12-26,2.31,2.33,2.24,2.34,22631.00,2.33,4.00,17.00,...,0.01,120557.80,0.06,0.00,0.86,2.31,2.33,0.12,5.91,0.40
402436,76858.00,2019-12-27,2.11,2.15,2.09,2.31,120340.00,2.15,4.00,17.00,...,0.02,52730.23,-0.08,0.00,1.88,2.11,2.15,0.05,-7.73,0.42
402456,76858.00,2019-12-30,2.05,2.06,2.05,2.15,68403.00,2.06,4.00,17.00,...,0.00,258731.01,-0.04,0.00,0.49,2.05,2.06,0.26,-4.42,0.40


#### Investigating Zero Volume

In [56]:
bid_ask_df[bid_ask_df.VOL == 0]

,PERMNO,DATE,BID,ASK,BIDLO,ASKHI,VOL,PRC,decile_category,stock_num,...,REL_SPREAD,DLR_VOL,DLY_RETURN,DLY_VARIANCE,Quote Spread (%),Bid ($),Ask ($),Volume ($million),Daily Return (%),Return Variance (%)
424,15203.00,2000-02-02,3.62,4.00,3.62,4.00,0.00,-3.81,1.00,5.00,...,0.10,-0.00,0.00,0.00,9.84,3.62,4.00,-0.00,0.00,0.13
444,15203.00,2000-02-03,3.62,4.00,3.62,4.00,0.00,-3.81,1.00,5.00,...,0.10,-0.00,0.00,0.00,9.84,3.62,4.00,-0.00,0.00,0.13
447,32062.00,2000-02-03,2.50,2.75,2.50,2.75,0.00,-2.62,1.00,8.00,...,0.10,15000.00,0.05,0.01,9.52,2.50,2.75,0.01,5.00,0.88
464,15203.00,2000-02-04,3.62,4.00,3.62,4.00,0.00,-3.81,1.00,5.00,...,0.10,-0.00,0.00,0.00,9.84,3.62,4.00,-0.00,0.00,0.13
467,32062.00,2000-02-04,2.50,2.75,2.50,2.75,0.00,-2.62,1.00,8.00,...,0.10,-0.00,0.00,0.01,9.52,2.50,2.75,-0.00,0.00,0.88
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
814954,75470.00,2002-09-27,8.90,8.99,8.90,8.99,0.00,-8.94,9.00,20.00,...,0.01,98790.00,0.01,0.00,1.01,8.90,8.99,0.10,0.51,0.04
815014,75470.00,2002-10-02,8.91,9.00,8.91,9.00,0.00,-8.95,9.00,20.00,...,0.01,30906.00,-0.01,0.00,1.01,8.91,9.00,0.03,-1.49,0.04
815153,75470.00,2002-10-11,8.36,8.49,8.36,8.49,0.00,-8.42,9.00,20.00,...,0.02,12555.00,0.01,0.00,1.54,8.36,8.49,0.01,0.66,0.05
829048,13507.00,2005-10-17,112.56,113.26,112.56,113.26,0.00,-112.91,9.00,6.00,...,0.01,54509.15,0.00,0.00,0.62,112.56,113.26,0.05,0.46,0.02
